In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt

import re
import sys
sys.path.append("../")
from functools import partial

from utils.dataset import OCRDataset
from models.generator import JAMOSeq2SeqGenerator
from models.jamo import compose_unicode
from tensorflow.keras.layers import Layer

import tensorflow as tf 
assert int(tf.__version__[:1]) < 2.0, "해당 코드는 1.x에서만 동작합니다."
tf.logging.set_verbosity(tf.logging.ERROR)

## 한글 Matplotlib 출력 세팅

In [ ]:
# 1. 나눔 폰트를 다운받기
!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.font_manager as fm
# 2. 나눔 폰트의 위치 가져오기 
system_font = fm.findSystemFonts() # 현재 시스템에 설치된 폰트
nanum_fonts = [font for font in system_font if "NanumBarunGothic.ttf" in font]
font_path = nanum_fonts[0] # 설정할 폰트의 경로

# 3. 나눔 폰트로 설정하기
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc("font",family=font_name)

# 4. 폰트 재설정하기
fm._rebuild()

# 5. (optional) minus 기호 깨짐 방지
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

# \[ OCR 데이터셋 - SRN ver3.0 \]

MNIST 데이터셋을 통해 `SRN(Sequence Recognition Network)`을 학습시켜보도록 하겠습니다. SRN은 CRNN의 구조와 Seq2Seq, 그리고 Attention Network을 합친 모델입니다.

## 1. 데이터 가져오기 

In [ ]:
# 단어 최소/최대 길이
min_words = 4
max_words = 8

df = pd.read_csv("../datasets/wordslist.txt",names=['word'])
df = df.drop_duplicates()
df = df[df.word.str.match(r'^[가-힣]+$')]
df = df[
    df.word.map(
        lambda x: (len(x) >= min_words) 
        and (len(x) <= max_words))]
words = df.word.values
np.random.shuffle(words)
words = np.append(words,"정보화진흥원")

In [ ]:
# OCRDataset setting
OCRDataset = partial(OCRDataset,
                     font_size=24,
                     bg_noise=0.2,
                     affine_noise=(0.0,0.01),
                     normalize=False,
                     random_shift=False,                     
                     color_noise=(0.1,0.3),
                     gray_scale=True)

dataset = OCRDataset(words)
images, labels = dataset[0:3]

for image, label in zip(images, labels):
    plt.title(label)
    plt.imshow(image[:,:,0],cmap='gray')
    plt.show()

## 2. 데이터 Generator 가져오기

In [ ]:
# 단어 군을 기준으로 나누기 
# validation words는 5%만 둚
valid_words = words[:len(words)*5//100]
train_words = words[len(words)*5//100:]
train_words = train_words

valid_set = OCRDataset(valid_words)
train_set = OCRDataset(train_words)

train_gen = JAMOSeq2SeqGenerator(train_set, batch_size=64)
valid_gen = JAMOSeq2SeqGenerator(valid_set, batch_size=64)

In [ ]:
X,Y = train_gen[0]
for i in range(3):
    image = X['images'][i,:,:,0]
    plt.title(compose_unicode(Y['output_seqs'][i])[0])
    plt.imshow(image,cmap='gray')
    plt.show()

> 모델을 학습할 때에는 Images, Decoder Input, Model Output 이렇게 총 3개의 데이터가 필요합니다. Decoder input과 Model Output은 1번의 Time Step 만큼 차이가 납니다. 

----

# 3. 모델 구성하기



In [ ]:
from models.layers import Map2Sequence, ResidualConvFeatureExtractor
from models.layers import BLSTMEncoder, CTCDecoder, DotAttention
from models.layers import JamoEmbedding, JamoCompose, JamoClassifier

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Conv1D

In [ ]:
class SummarySequence(Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.summary_conv_1 = Conv1D(units, 3, 2,
                                     activation='relu',
                                     padding='same')
        self.summary_conv_2 = Conv1D(units, 3, 2,
                                     activation='relu',
                                     padding='same')
        self.summary_conv_3 = Conv1D(units, 3, 2,
                                     activation='relu',
                                     padding='same')
        self.built = True

    def call(self, inputs, **kwargs):
        x = self.summary_conv_1(inputs)
        x = self.summary_conv_2(x)
        x = self.summary_conv_3(x)        
        return x
    
    def get_config(self):
        config = {
            "units": self.units
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items())) 

In [ ]:
K.clear_session()

height = 36

n_conv = 16 # the number of Convolution filter
n_state = 128 # the number of GRU units
n_embed = 16 # the size of embedding vector
n_summary = 64 # the size of summary vector
n_clf = 256 # the number of units in classifier Dense layer

# Image Encoder
inputs = Input(shape=(height, None, 1),name='images')
conv_maps = ResidualConvFeatureExtractor(n_conv,
                                         name='feature_extractor')(inputs)
feature_seqs = Map2Sequence(name='map_to_sequence')(conv_maps)

# Image Summary
summary_seqs = SummarySequence(n_summary)(feature_seqs)
summary_gru_layer = GRU(n_state, 
                name='summary_gru',
                go_backwards=True,
                return_state=True)
summary_state = summary_gru_layer(summary_seqs)[0]

# Embedding Layer
decoder_inputs = Input(shape=(None,), name='decoder_inputs')
jamo_embedding_layer = JamoEmbedding(n_embed)
jamo_embedding = jamo_embedding_layer(decoder_inputs)

# Text Decoder
#decoder_state_inputs = Input(shape=(n_state,), name='decoder_state')
gru_layer = GRU(n_state, 
                name='decoder_gru', 
                return_sequences=True)
states_decoder = gru_layer(jamo_embedding,
                           initial_state=summary_state)

# Attention Layer
dotattend = DotAttention(n_state=n_state)
context, attention = dotattend([feature_seqs, states_decoder])

concat_state = concatenate([context, states_decoder],
                            axis=-1, name='concat_output')
# Classification Layer
jamo_clf = JamoClassifier(n_clf, name='output_seqs')

predictions = jamo_clf(concat_state)

## (8) 추론 모델과 학습 모델 구성하기

In [ ]:
# For training
trainer = Model([inputs, 
                 decoder_inputs], 
                 predictions, name='trainer')

# For Inference
# - (1) Encoder
encoder = Model(inputs, [feature_seqs,summary_state], 
                name='encoder')

# - (2) Decoder
num_feature = feature_seqs.shape.as_list()[-1]
states_encoder_input = Input((None, num_feature),
                             name='states_encoder_input')
decoder_state_inputs = Input(shape=(n_state,), name='decoder_state')

states_decoder = gru_layer(jamo_embedding,
                           initial_state=decoder_state_inputs)

context, attention = dotattend([states_encoder_input, states_decoder])
concat_state = concatenate([context, states_decoder], axis=-1, 
                            name='concat_output')
predictions = jamo_clf(concat_state)
output_decoder = JamoCompose(name='jamocompose')(predictions)

decoder = Model([states_encoder_input,
                 decoder_inputs,
                 decoder_state_inputs], 
                [states_decoder, output_decoder], name='decoder')

## (9) 학습 모델 Compile하기

In [ ]:
from models.optimizer import AdamW
from models.losses import jamo_categorical_crossentropy

In [ ]:
y_true = tf.placeholder(shape=(None, None), dtype=tf.int32)

trainer.compile(AdamW(lr=1e-2),
                loss={
                    "output_seqs":jamo_categorical_crossentropy,
                },
                target_tensors=[y_true])

## (10) 모델 학습하기

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

callbacks =[]
rlrop = ReduceLROnPlateau(
    factor=0.5, patience=2, 
    min_lr=1e-6, verbose=1)
callbacks.append(rlrop)

epochs = 50

In [ ]:
train_gen = JAMOSeq2SeqGenerator(train_set, batch_size=32,
                                 return_initial_state=False,
                                 state_size=n_state)
valid_gen = JAMOSeq2SeqGenerator(valid_set, batch_size=32,
                                 return_initial_state=False,                                 
                                 state_size=n_state)

In [ ]:
hist = trainer.fit_generator(train_gen,
                             steps_per_epoch=1000,                             
                             epochs=epochs,
                             validation_data=valid_gen,
                             callbacks=callbacks)

## (11) 모델 평가하기

In [ ]:
EOS_TOKEN = train_gen.num_classes

In [ ]:
X,_ = test_gen[0]

# Target image 
target_images = X['images'][:10]

# Encoder 결과 계산
states_encoder_ = encoder.predict(target_images)

In [ ]:
# Decoder
batch_size = target_images.shape[0]

prev_inputs = np.ones((batch_size,1)) * EOS_TOKEN
prev_states = np.zeros((batch_size, 512))

In [ ]:
result = prev_inputs.copy()
while True:
    states_decoder_, predictions_ = decoder.predict({
        "states_encoder_input" : states_encoder_,
        "decoder_inputs": prev_inputs,
        "decoder_state": prev_states        
    })
    prev_states = states_decoder_[:,-1,:]
    prev_inputs = np.argmax(predictions_,axis=-1)
    
    if np.all(prev_inputs == EOS_TOKEN):
        break
    result = np.concatenate([result,prev_inputs],axis=-1)
result = result[:,1:].squeeze()

In [ ]:
for image, seq in zip(target_images,result):
    plt.title(seq)
    plt.imshow(image[:,:,0])
    plt.show()


Reference : 
1. [Attention based Seq2Seq in Keras](https://wanasit.github.io/attention-based-sequence-to-sequence-in-keras.html)
2. [](https://medium.com/datalogue/attention-in-keras-1892773a4f22)